In [1]:
# this entire code takes in recorded video as input, then it processes them, run detecting & tracking software,
# generate a table of speed against time & a video of graph being drawn in real time

import os
import subprocess
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.animation as animation
from DetectTrackUtilsMul import *
from TimeUtils import *

In [2]:
yolo_model = "/Users/yapginehann/Desktop/codes/train/results/weights/best.pt"

# video files path
vid_directory = "/Users/yapginehann/Desktop/codes/recordings/ex"
streamed_vid = f"{vid_directory}/ex.mp4"
processed_vid = f"{vid_directory}/ex - p.mp4"
output_vid = f"{vid_directory}/ex - d.mp4"
output_dot_vid = f"{vid_directory}/ex - dd.mp4"
output_table = f"{vid_directory}/ex - r.csv"
output_graph = f"{vid_directory}/ex - r.png"
output_graph_vid = f"{vid_directory}/ex - r.mp4"

In [3]:
# process video so that it plays for 2 minutes in realtime
def process_vid(pre_vid, post_vid):
    subprocess.run(["ffmpeg", "-i", f"{pre_vid}", "-ss", "0:00:00", "-t", "00:02:00", "-filter:v", "setpts=1.25*PTS", f"{post_vid}"]) 

# generate graph, output mean_activity
def generate_graph_and_mean(table, output_graph):
    df = pd.read_csv(table)

    plt.plot(df['Time'], df['Aggregate Velocity'], color='red', label='Velocity (Rolling)')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Velocity (pixels per second)')
    plt.title(f"Fishes' velocity")
    plt.axhline(y = df['Aggregate Velocity'].mean(), color = 'darkred', label = 'Average Velocity')
    plt.legend()
    # plt.ylim(None, 1500)
    plt.savefig(output_graph) 

    mean_activity = df['Aggregate Velocity'].mean()
    return mean_activity 

In [ ]:
process_vid(streamed_vid, processed_vid)

In [ ]:
fish_detector_tracker(processed_vid, output_vid, output_dot_vid, output_table, yolo_model)

In [4]:
mean_activity = generate_graph_and_mean(output_table, output_graph)

In [5]:
df = pd.read_csv(output_table)

fig, ax = plt.subplots()

# Create empty line objects for each category of data
lines = []

# Define the line objects
line, = ax.plot([], [], label='velocity', color='red')
lines.append(line)

# Set axis labels and title
ax.set_xlabel('Time (seconds)')
ax.set_ylabel('Velocity (pixels per second)')
ax.set_title('Velocity of fishes when feeding')
ax.set_ylim(500,4000)
ax.legend()

# Define the update function
def update(frame):
    # Load the data for the current frame
    data = df.iloc[:frame+1]

    # Update the data for the line object
    x = data['Time']
    y = data['Aggregate Velocity']
    lines[0].set_data(x, y)

    # Set the x-axis limit to show only the most recent data
    ax.set_xlim(max(0, x.iloc[-1] - 60), x.iloc[-1])

In [6]:
# output video of graph being drawn in realtime 
ani = animation.FuncAnimation(fig, update, frames=len(df), interval=100)
FFwriter = animation.FFMpegWriter(fps=16)
ani.save(output_graph_vid, writer=FFwriter)

/var/folders/tk/7mv5n79j5xl_gyv7hrpnw3w40000gn/T/ipykernel_65890/488422229.py:30: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax.set_xlim(max(0, x.iloc[-1] - 60), x.iloc[-1])
